In [2]:
# import statements

import numpy as np
import numpy.random as ra
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
import colorcet as cc

In [20]:
# Set N = side-length, F = number of frames, I = iterations per frame, L = last completed frame, and S = update density
N = 512
F = 500
I = 200
L = -1
S = 0.05

In [39]:
# Define auxiliary functions (fast)

theta = None

def InsideCircle(i,j):
    return (i+0.5-0.5*N)**2 + (j+0.5-0.5*N)**2 <= (0.5*N - 16)**2

def CircleBC(i,j):
    if InsideCircle(i,j):
        return theta[i,j]
    else:
        return np.arctan2(i+0.5-0.5*N, j+0.5-0.5*N)

adjacency = [(0,1),(0,-1),(1,0),(-1,0)]

def Hloc(i,j,proptheta=None):
    if proptheta is None:
        proptheta = CircleBC(i,j)
    return - sum([np.cos(proptheta - CircleBC(i+a,j+b)) for (a,b) in adjacency])

def MetropolisUpdate(i,j,beta=1.1343):
    proptheta = ra.uniform(-np.pi,np.pi)
    if ra.uniform(0,1) < np.exp(- beta * (Hloc(i,j,proptheta) - Hloc(i,j))):
        theta[i,j] = proptheta


beta = lambda x : 1.1343 + 15 * (1+100*x**6) * (x-1/3)**3

In [19]:
# Run simulation (slow)

if L > -1:
    theta = np.load(f'hexdata/{L:06d}.npy').reshape((N,N))
else:
    theta = ra.uniform(-np.pi,np.pi,size=(N,N))
    for i in range(N):
        for j in range(N):
            theta[i,j] = CircleBC(i,j)

for frame in tqdm(range(L+1, F)):
    b = beta(frame/F)

    for _ in range(I):
        to_update = ra.uniform(0,1,size=(N,N)) < 0.05

        for i in range(N):
            for j in range(N):
                if InsideCircle(i,j) and to_update[i,j]:
                    MetropolisUpdate(i,j,beta=b)

    np.save(f'circledata/{frame:06d}.npy',theta)

 90%|█████████ | 27/30 [00:08<00:00,  3.01it/s]/var/folders/k5/czr0d7xn2w5456rcmtfxtnm80000gn/T/ipykernel_54210/1700150502.py:23: RuntimeWarning: overflow encountered in exp
  if ra.uniform(0,1) < np.exp(- beta * (Hloc(i,j,proptheta) - Hloc(i,j))):
100%|██████████| 30/30 [00:09<00:00,  3.00it/s]


In [21]:
# Generate images for video (slow)

thetas = [np.load(f'circledata/{i:06d}.npy').reshape((N,N)) for i in range(L+1,F)]

for frame in tqdm(range(L+1,F)):
    plt.imsave(f'circleimages/{frame:06d}.png', thetas[frame-L-1], format='png', cmap=cc.m_cyclic_rygcbmr_50_90_c64, vmin=-np.pi, vmax=np.pi)

100%|██████████| 500/500 [00:38<00:00, 12.87it/s]


In [19]:
# COLOR TESTING

theta = np.load(f'circledata/002999.npy').reshape((N,N))
plt.imsave(f'circleimages/colortest.png', theta, format='png', cmap=cc.m_cyclic_rygcbmr_50_90_c64, vmin=-np.pi, vmax=np.pi)